In [1]:
!pip install BeautifulSoup4
#
# import libraries
from bs4 import BeautifulSoup
import urllib.request
import csv
from urllib.request import urlopen
import re
#-----------------------------------------------------------------------------    
def process(urlpage):
# Output line
    line=[]
    line2=[]
# query the website and return the html to the variable 'page'
    page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
    soup = BeautifulSoup(page, 'html.parser')


#
#    pubmedDOI
# https://stackoverflow.com/questions/59333559/how-to-implement-if-statement-with-beautifulsoup-and-selenium - for below if loop
    literature = soup.find_all('li', id='pubmedDOI')
    if literature:
        #    Find doi after 4'th > and first < after that 
        doi=get_item(str(literature[0]),4,0)
#         print("doi",doi)
    else:
        doi = ""
    
#
#    pubmedLinks
# https://stackoverflow.com/questions/59333559/how-to-implement-if-statement-with-beautifulsoup-and-selenium - for below if loop
    pubmedlinks = soup.find_all('li', id='pubmedLinks')
    if pubmedlinks:
        #    Find doi after 4'th > and first < after that 
        pubmedno=get_item(str(pubmedlinks[0]),4,0)
#         print("pubmed ID", pubmedno)
    else:
        pubmedno = ""
    


#
#    Classification
#
    classification = soup.find_all('li', id='header_classification')
 #  After 3rd ">" and the "<" after that 
    classif=get_item(str(classification[0]),3,0)
#     print("Classification", classif)

    

#    Organism
#
#
    organism = soup.find_all('li', id='header_organism')
    
    searchorganism=str(organism[0]) # this is what we willserach of organisms.
    organisms_list=""   # Empty list for results
#   There can be more than one organism, which starts after 'rcsb_entity_source_organism.taxonomy_lineage.name:
#   and ends after the first "">
    organism_tag='rcsb_entity_source_organism.taxonomy_lineage.name:'
#    Here we find how many organisms there are and where they start in the strimg (in to list "starts")
    searchlen=len(organism_tag) # Lenth of search tag that we have to jump over to find organism
 #  Find start (a list) position in string of serach tag  
    starts =[i for i in range(len(searchorganism)) if searchorganism.startswith
             (organism_tag, i)]
#   Here we loop over the starting positions and find the ">" in each, put in "ends" where end[0] is the first
    for j in range(len(starts)):
        ends =[i for i in range(len(searchorganism[starts[j]+searchlen:len(searchorganism)]))if searchorganism[starts[j]+searchlen:len(searchorganism)].startswith(">", i)]
#          Write out from start position at of search tag to the ends[0] after that, minus 1, to avoid the" 
        organisms_list=organisms_list+ " " + searchorganism[starts[j]+searchlen:starts[j]+searchlen+ends[0]-1]
#         print(organisms_list)

  
    
#
#    Method
#
    ExpMethod = soup.find_all('li', id='exp_header_0_method')

    expMethod=get_item(str(ExpMethod[0]),3,0)
#     print("Experimental Method",expMethod)
    line.append([code,doi, pubmedno,classif,expMethod,organisms_list])
    return line

    
def get_item(string,no_greater,no_less):
#-----------------------------------------------------------------------------
# Parse out item between no_greater ">" and no_less "<" after that.
#  Split on ">" up to no_greater 
#    print("string", string)
    found_item=string.split(">", no_greater)
    return found_item[no_greater].split("<")[no_less]

def get_li_item(soup,id_str,no_greater,no_less):
#-----------------------------------------------------------------------------
#
#   li item. With ide=id_stream. Parse itemb etween no_greater ">"'s and no_less "<"'s after that.
#
# <li id="header_organism"><strong>Organism(s):&nbsp</strong><a href="/search?q=rcsb_entity_source_organism.taxonomy_lineage.name:Sus scrofa">Sus scrofa</a></li>
#
    li_str = soup.find_all('li', id=id_str)
 #  Split on ">" up to third occurrence 
    li_item=str(li_str[0]).split(">", no_greater)[no_greater]
#   Split before first occurence of "<"  
#    print("OrganismsAnswer",li_item.split("<")[no_less])
    li_item=li_item.split("<")[no_less]
    return li_item

def get_fromtable_column(soup,id_str,skip,column,no_greater,no_less,code):
#
#   Get table column from table with id=id_str. Skip ths number of rows
#   Table rom Column in teh rows
#   Parse item between no_greater ">"'s and no_less "<"'s after that. 
    table1 = soup.find('table', id=id_str)
#    print("Table1",table1)
# Ugly - has two ligandsmaintable. One for oligos and one ro small molecuels.Only one - easier to just make
# and exception since it was the only one out of 95. The "Oligosacharides" and "Small Molecules" are both 
# in <div> tags. Better to find these and then sift ligandstable within.
    if (table1 and code!="3b08"):   
        tr1 = table1.findAll(['tr'])
        listitems=""
        for item in tr1[skip:]:
            table_column = item.findAll('td')[column].contents
            singleitem=str(table_column).split(">", no_greater)[no_greater]
            singleitem=singleitem.split("<",1)[no_less]
            listitems=listitems+" "+singleitem

    else:
        listitems="None"
    return listitems
#
def get_fromtable_column_row(soup,id_str,skip,column,row,no_greater,no_less):
#
#   Get table column from table with id=id_str. Skip ths number of rows
#   Table from Column in the rows
#   Parse item between no_greater ">"'s and no_less "<"'s after that. 
    table1 = soup.find('table', id=id_str)
    tr1 = table1.findAll(['tr'])
    singleitem=""
    table_column = tr1[row].findAll('td')[column].contents

    singleitem=str(table_column)
    return singleitem


def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result


#
# Main 
#
# Take  a list of pdb code. Loop over 'https://www.rcsb.org/structure/'+str(code) to find data


csvline=[]  #This is what will be written out

# Read in list of codes


with open('C:/unannotated_pdb_list.csv') as csvfile:   # Open csv file
#with open('complement.csv') as csvfile:   # Open csv file
    readCSV = csv.reader(csvfile, delimiter=',') # Read csv file using a comma a delimiter
    for code in readCSV: # Loop over rows in csv file


        code = "".join(code)  # Takes out brackets and quotes to give clean pdbcode string
        print("Code is",code)
        urlpage ='https://www.rcsb.org/structure/'+str(code)
        print ("urlpage = ", urlpage)

#         numb=process(urlpage)   # Get data from page. Set rows and columns in process function
        csvline=csvline+process(urlpage)  
#         print("Process ",numb)
csvfile.close()

# print("line",csvline)

# Write to csv file.
with open('C:/unannotated_pdb_info.csv', 'w') as csvFile:
#with open('testop.csv', 'w') as csvFile:
# https://stackoverflow.com/questions/19663413/python-beautifulsoup-empty-rows-in-csv-file - for removing empty lines from writing in the file
    writer = csv.writer(csvFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator = "\n")
    writer.writerow(['Code','doi','Pubmed ID', 'Classification','ExpMethod','Organism(s)'])

    for i in csvline:
        writer.writerow(i)
csvFile.close()



Code is 3O6O
urlpage =  https://www.rcsb.org/structure/3O6O
Code is 3OMU
urlpage =  https://www.rcsb.org/structure/3OMU
Code is 6HIV
urlpage =  https://www.rcsb.org/structure/6HIV
Code is 6HIX
urlpage =  https://www.rcsb.org/structure/6HIX
Code is 6YXX
urlpage =  https://www.rcsb.org/structure/6YXX
Code is 6YXY
urlpage =  https://www.rcsb.org/structure/6YXY
Code is 4BI9
urlpage =  https://www.rcsb.org/structure/4BI9
Code is 4BIA
urlpage =  https://www.rcsb.org/structure/4BIA
Code is 4BP8
urlpage =  https://www.rcsb.org/structure/4BP8
Code is 4BP9
urlpage =  https://www.rcsb.org/structure/4BP9
Code is 1OEP
urlpage =  https://www.rcsb.org/structure/1OEP
Code is 2PU1
urlpage =  https://www.rcsb.org/structure/2PU1
Code is 2MEK
urlpage =  https://www.rcsb.org/structure/2MEK
Code is 6LP1
urlpage =  https://www.rcsb.org/structure/6LP1
Code is 7AOI
urlpage =  https://www.rcsb.org/structure/7AOI
Code is 4V8M
urlpage =  https://www.rcsb.org/structure/4V8M
Code is 3K7U
urlpage =  https://www.rcsb